In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from datetime import date
today = date.today().strftime("%d-%m-%Y")
df = pd.read_csv("/Users/bhargavmukhami/Desktop/Stock_Project/EQUITY_L.csv")
request_per_minute = 40
delay = 60 / request_per_minute
count = 0
for i in df['SYMBOL']:
    print(count, i)
    try:
        link = "https://www.screener.in/company/" + i + "/consolidated/"
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        name = soup.find_all(class_ = "name")
        list_name = []
        for j in name:
            list_name.append(j.get_text())
        cleaned_list = ["SYMBOL"]
        cleaned_list.extend([item.strip() for item in list_name])
        values = soup.find_all(class_ = "number")
        list_values = [i]
        for j in values:
            list_values.append(j.get_text())
        tally = 0
        for j in list_values[1:]:
            if j != '':
                tally = 1
        if tally == 0:
            link = "https://www.screener.in/company/" + i
            page = requests.get(link)
            soup = BeautifulSoup(page.content, 'html.parser')
            name = soup.find_all(class_ = "name")
            list_name = []
            for j in name:
                list_name.append(j.get_text())
            cleaned_list = ["SYMBOL"]
            cleaned_list.extend([item.strip() for item in list_name])
            values = soup.find_all(class_ = "number")
            list_values = [i]
            for i in values:
                list_values.append(i.get_text())
        high_low = list_values[3] + "/" + list_values[4]
        list_values[3] = high_low
        for i in range(4, 10):
            list_values[i] = list_values[i + 1]
        list_values.pop()
        stock_info = dict(zip(cleaned_list, list_values))
        np = soup.find_all('tr', class_ = "strong")
        soup1 = BeautifulSoup(str(np[2]), 'html.parser')
        np_quarterly = [td.text for td in soup1.find_all('td') if (td.text.isdigit() or td.text[0] == '-')]
        soup2 = BeautifulSoup(str(np[5]), 'html.parser')
        np_yearly = [td.text for td in soup2.find_all('td') if (td.text.isdigit() or td.text[0] == '-')]
        stock_info["np_quarterly"] = np_quarterly
        stock_info["np_yearly"] = np_yearly
        cg = soup.find_all('table', class_ = "ranges-table")
        new_cg = str(cg).replace('[', '').replace(']', '').split(', ')
        def extract_ttm(html):
            soup_cg = BeautifulSoup(html, 'html.parser')
            table_rows = soup_cg.find_all('tr')
            for row in table_rows:
                cells = row.find_all('td')
                for cell in cells:
                    if cell.text.strip() == 'TTM:':
                        ttm_value = cell.find_next('td').text.strip()
                        return ttm_value
        sales_growth_ttm = extract_ttm(new_cg[0])
        profit_growth_ttm = extract_ttm(new_cg[1])
        stock_info["Compounded Sales Growth"] = sales_growth_ttm
        stock_info["Compounded Profit Growth"] = profit_growth_ttm
        stripe = soup.find_all('tr', class_ = "stripe")
        soup_stripe = BeautifulSoup(str(stripe), 'html.parser')

        rows = soup_stripe.find_all('tr')

        cwip_values = []

        for row in rows:
            if 'CWIP' in row.get_text():
                cwip_values = [cell.get_text().strip() for cell in row.find_all('td')[1:]]
                break

        stock_info["CWIP"] = cwip_values



        cash_operating_values = []

        for row in rows:
            if 'Cash from Operating Activity' in row.get_text():
                cash_operating_values = [cell.get_text().strip() for cell in row.find_all('td')[1:]]
                break 

        stock_info["Cash from Operating Activities"] = cash_operating_values

        debtor_days = []

        for row in rows:
            if 'Debtor Days' in row.get_text():
                debtor_days = [cell.get_text().strip() for cell in row.find_all('td')[1:]]
                break

        stock_info["Debtor Days"] = debtor_days
        
        shrp = soup.find_all('tr')
        soup_shrp = BeautifulSoup(str(shrp), 'html.parser')
        new_shrp = str(soup_shrp).replace('[', '').replace(']', '').split(', ')
        html_snippets = new_shrp[-12:]
        promoters = []
        fiis = []
        diis = []
        public = []

        for snippet in html_snippets:
            soup = BeautifulSoup(snippet, 'html.parser')
            percentages = [td.get_text(strip=True) for td in soup.find_all('td') if '%' in td.get_text(strip=True)]
            
            if 'Promoters' in snippet:
                promoters.extend(percentages)
            elif 'FIIs' in snippet:
                fiis.extend(percentages)
            elif 'DIIs' in snippet:
                diis.extend(percentages)
            elif 'Public' in snippet:
                public.extend(percentages)
        stock_info["Promoter holding"] = promoters
        stock_info["FIIs"] = fiis
        stock_info["DIIs"] = diis
        stock_info["Public"] = public
        print(stock_info)
        print()
        with open(f"stock_data_fundamental/stock({today}).csv", mode="a") as csvfile:
            fieldnames = stock_info.keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            if csvfile.tell() == 0:
                writer.writeheader()
            writer.writerow(stock_info)
        time.sleep(delay)
        count += 1
    except:
        count += 1
        continue


0 20MICRONS
{'SYMBOL': '20MICRONS', 'Market Cap': '794', 'Current Price': '225', 'High / Low': '348/152', 'Stock P/E': '12.4', 'Book Value': '122', 'Dividend Yield': '0.54', 'ROCE': '19.2', 'ROE': '16.3', 'Face Value': '5.00', 'np_quarterly': ['11', '13', '12', '7', '9', '15', '16', '12', '14', '18', '16', '13', '15'], 'np_yearly': ['-2', '-2', '11', '16', '19', '25', '24', '23', '35', '42', '56', '62'], 'Compounded Sales Growth': '17%', 'Compounded Profit Growth': '11%', 'CWIP': ['9', '7', '4', '7', '8', '4', '11', '3', '5', '8', '12', '9'], 'Cash from Operating Activities': ['37', '30', '25', '44', '61', '53', '45', '52', '35', '63', '54', '32'], 'Debtor Days': ['65', '51', '57', '56', '57', '53', '57', '68', '60', '51', '59', '57'], 'Promoter holding': ['44.30%', '43.36%', '44.03%', '44.83%', '44.83%', '44.83%', '44.83%', '44.95%', '45.04%'], 'FIIs': ['0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '1.07%', '0.93%', '0.73%', '0.84%'], 'DIIs': ['0.03%', '0.37%', '0.11%', '0.00%', '0.00%